In [1]:
import pandas as pd
import json
import nltk
from nltk.stem import WordNetLemmatizer
import re
import os

nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     /home/chaimaa.abi/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/chaimaa.abi/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
df = pd.read_json("/home/chaimaa.abi/Desktop/AI_project/layer1.json")

In [3]:
lemmatizer = WordNetLemmatizer()
ignore_words = [
   'easy','recipe',"really","breakfast","quick","foolproof","the","lunch","simple","comfort","lovers",
    "friendly","weeknight","time","miracle","best","everything","heaven","baba","boy","dirt","bliss",
    "wonderful","momma","fresh","version","just","thumbprint","preserved","lover","tips","city","colorful",
    "your","bill","joy","treat","substitute", "impossible","shortcut","company","market","poor","style","new","elegent",
    "refreshing","dessert","better","killer","game","sunrise","night","fool","fantastic","uncle","basic","almost","original",
    "another","mrs","fabulous","speedy","mom","moms","mother","granny","grandmas","our","secret","fall","crazy","dirty",
    "ingredient","fix","copycat","country","birthday","dad","simply","easiest","chef","lazy","grandma","aunt","authentic","mama",
    "supper","classic","healthy","healthier","heavenly","magic","slow","ww","favorite","great","ii","iii","iv","yummy","tasty",
    "party","ultimate","ever","halloween","diabetic","very","day","old","awesome","thanksgiving","amazing","famous","morning","my",
    "fashioned","holiday","oldfashioned","dinner","minute","surprise","food","good","special","key"
]

In [4]:
def remove_unnecessary_words_and_characters(title):
    new=[]
    for word in title.split():
        if len(word)>1 and word not in new and word not in ignore_words:
            new.append(word)
    return " ".join(new)

def clean_title(text):   
    text = text.lower()
    text =  remove_unnecessary_words_and_characters(text)

    words = nltk.word_tokenize(text)  # Tokenize the sentence into words
    lemmatized_words = [lemmatizer.lemmatize(word) for word in words]  # Lemmatize each word
    text = " ".join(lemmatized_words)  # Join the words back into a sentence
    
    text=re.sub(r'[:\*+,-/:;<=>@\\^_`(){|}~]+',' ',text)
    text=text.replace(r'&amp;?',r' and ')
    text=text.replace(r'amp', r' and ')

    text = re.sub(r'[^a-zA-Z]', ' ', text)  # Remove non-alphabetic characters
    text = re.sub(r'\s+', ' ', text).strip()

    # Remove trailing whitespace
    text = re.sub(r'\d+', '', text)
    return text

In [5]:
df['sub_col'] = df.title.apply(lambda x:len(str(x).split()))
small_title_df = df[df["sub_col"]<=4]
small_title_df['clean_title'] = small_title_df.title.apply(clean_title)

/tmp/ipykernel_1045606/3238442816.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  small_title_df['clean_title'] = small_title_df.title.apply(clean_title)


In [ ]:
# image_names = pd.read_json("/home/chaimaa.abi/Downloads/layer2.json")
# joined_df = pd.merge(small_title_df, image_names, on='id')
# new_train_part = joined_df[joined_df["partition"]=="train"]
# title_counts = new_train_part['title'].value_counts()

# new_train_part['count'] = new_train_part['title'].map(title_counts)
# df_sorted = new_train_part.sort_values(by='count', ascending=False)
# df_sorted = df_sorted.reset_index(drop = True)

In [6]:
image_names = pd.read_json("/home/chaimaa.abi/Desktop/AI_project/layer2.json")

In [7]:
joined_df = pd.merge(small_title_df, image_names, on='id')
# new_train_part = joined_df[joined_df["partition"]=="train"]
title_counts = joined_df['title'].value_counts()

joined_df['count'] = joined_df['title'].map(title_counts)
df_sorted = joined_df.sort_values(by='count', ascending=False)
df_sorted = df_sorted.reset_index(drop = True)

In [13]:
# we will keep only 1233 food classes 
df_sorted = df_sorted[df_sorted['count']>7]

In [14]:
# Create annotation jsons 
train_ann = []
for ind in df_sorted.index:
    for each in df_sorted['images'][ind]:
        temp_dict = {}
        temp_dict['caption'] = df_sorted['clean_title'][ind]
        img_id = each['id']
        img_path = os.path.join(df_sorted['partition'][ind], "{}/{}/{}/{}/{}".format(img_id[0], img_id[1], img_id[2], img_id[3], img_id))
        temp_dict['image'] = img_path
        train_ann.append(temp_dict)

In [15]:
FILE_PATH = '/home/chaimaa.abi/Desktop/AI_project/all_162_21.json'

with open(FILE_PATH, 'w') as output_file:
	json.dump(train_ann, output_file)